In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -U keras-tuner

In [1]:
import argparse
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.keras import layers, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.python.keras.models import load_model, Model
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import kerastuner as kt
from kerastuner.tuners import RandomSearch


def report_model(md, model_name, images, labels, classes):
    print("**********REPORT FOR ", model_name.upper(), " MODEL********** /n")
    predictions = md.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)
    report = classification_report(labels, predicted_labels, target_names=list(classes.keys()),
                                   labels=list(classes.values()))
    print(report)
    print("******************************************")

def build_model(hp):
    base_model = keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),
        pooling='avg'
    )

    # Fine-tuning
    for layer in base_model.layers[:249]:
        layer.trainable = False
    for layer in base_model.layers[249:]:
        layer.trainable = True

    x = base_model.output
    x = keras.layers.Dense(hp.Int('units', min_value=512, max_value=2048, step=512), activation='relu')(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1))(x)
    predictions = keras.layers.Dense(3, activation='softmax')(x)

    model = keras.models.Model(inputs=base_model.input, outputs=predictions)

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', min_value=1e-4, max_value=1e-2, sampling='log')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

class_map = {'eagle': 0, 'panda': 1, 'polar-bear': 2}

# Učitavanje slika i oznaka
train_imgs = np.load('drive/MyDrive/cropped/train_imgs.npy')
train_labels = np.load('drive/MyDrive/cropped/train_labels.npy')
test_imgs = np.load('drive/MyDrive/cropped/test_imgs.npy')
test_labels = np.load('drive/MyDrive/cropped/test_labels.npy')
val_imgs = np.load('drive/MyDrive/cropped/val_imgs.npy')
val_labels = np.load('drive/MyDrive/cropped/val_labels.npy')

print(f"train_imgs shape: {train_imgs.shape}")
print(f"test_imgs shape: {test_imgs.shape}")
print(f"val_imgs shape: {val_imgs.shape}")


# Augmentacija slika
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
)

# Kreiranje skupa podataka
train_set = train_datagen.flow(
    train_imgs,
    train_labels,
    batch_size=32,
    shuffle=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
)

val_set = val_datagen.flow(
    val_imgs,
    val_labels,
    batch_size=32,
    shuffle=False
)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
tuner = kt.GridSearch(
      build_model,
      objective='val_loss',
      max_trials=15)

tuner.search(train_set, epochs=5, validation_data=val_set)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Ispis hiperparametara
print("Best Hyperparameters:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")


Trial 2 Complete [00h 48m 55s]
val_loss: 0.4847310483455658

Best val_loss So Far: 0.33641353249549866
Total elapsed time: 01h 37m 08s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
512               |512               |units
False             |False             |dropout
0.00031623        |0.0001            |lr

Epoch 1/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 577s 7s/step - accuracy: 0.6166 - loss: 0.8103 - val_accuracy: 0.6244 - val_loss: 0.9396
Epoch 2/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 554s 7s/step - accuracy: 0.7823 - loss: 0.5252 - val_accuracy: 0.4520 - val_loss: 1.7178
Epoch 3/5
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8250 - loss: 0.4274

KeyboardInterrupt: 

In [ ]:
import argparse
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.keras import layers, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.python.keras.models import load_model, Model
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt


base_model = keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),
        pooling='avg'
    )

# Fine-tuning
for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

x = base_model.output
x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(3, activation='softmax')(x)


inception_model = keras.models.Model(inputs=base_model.input, outputs=predictions)

inception_model.compile(
optimizer=keras.optimizers.Adam(learning_rate=0.0002256574510240418),
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

history = inception_model.fit(train_set, epochs=50, validation_data=val_set, callbacks=[stop_early])
inception_model.save('inception_model.h5')

# Evaluacija modela
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)
test_set = test_datagen.flow(test_imgs, test_labels, batch_size=32, shuffle=False)
test_loss, test_accuracy = inception_model.evaluate(test_set)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

#Prikaz rezultata
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


#Confusion matrica
test_predictions = inception_model.predict(test_imgs)
test_predictions = np.argmax(test_predictions, axis=1)  # Convert probabilities to class labels

conf_matrix = confusion_matrix(test_labels, test_predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

report_model(inception_model, "inceptionv3", test_imgs, test_labels, class_map)

Epoch 1/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 61s 602ms/step - accuracy: 0.5476 - loss: 0.9243 - val_accuracy: 0.6976 - val_loss: 0.7222
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 65s 442ms/step - accuracy: 0.6594 - loss: 0.7410 - val_accuracy: 0.7041 - val_loss: 0.6480
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 42s 463ms/step - accuracy: 0.7083 - loss: 0.6591 - val_accuracy: 0.7366 - val_loss: 0.6057
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 36s 444ms/step - accuracy: 0.7321 - loss: 0.6078 - val_accuracy: 0.7675 - val_loss: 0.5794
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 40s 432ms/step - accuracy: 0.7372 - loss: 0.6089 - val_accuracy: 0.7659 - val_loss: 0.5671
Epoch 6/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 40s 427ms/step - accuracy: 0.7319 - loss: 0.5808 - val_accuracy: 0.8179 - val_loss: 0.4974
Epoch 7/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step - accuracy: 0.7763 - loss: 0.5270